<a href="https://www.kaggle.com/code/deephansda/vs-code-online?scriptVersionId=207927976" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
## Step 1: Install Necessary Tools
try:
    !pip install pyngrok
    !curl -fsSL https://code-server.dev/install.sh | sh
    !mkdir -p ~/.local/lib ~/.local/bin
    !curl -fL https://github.com/coder/code-server/releases/download/v$VERSION/code-server-$VERSION-linux-amd64.tar.gz \
      | tar -C ~/.local/lib -xz
    !mv ~/.local/lib/code-server-$VERSION-linux-amd64 ~/.local/lib/code-server-$VERSION
    !ln -s ~/.local/lib/code-server-$VERSION/bin/code-server ~/.local/bin/code-server
    !PATH="~/.local/bin:$PATH"
except Exception as e:
    print("Error installing tools:", e)

In [ ]:
import json
from threading import Timer
from queue import Queue
from pyngrok import ngrok

In [ ]:
import os

# Step 2: Detect Environment and Retrieve Tokens
try:
    # Get the current working directory
    current_dir = %pwd
    #enter ngrok_token here.
    ngrok_token = "" or None
    #enter ngrok_domain  here
    ngrok_domain = "" or None
    
    if ngrok_token and ngrok_domain:
        print("Ngrok token already exists. Skipping retrieval process.")
    else:
        if "/content" in current_dir:  # Google Colab
            try:
                from google.colab import userdata
                ngrok_token = userdata.get("NGROK_TOKEN")
                ngrok_domain = userdata.get("NGROK_DOMAIN")
                print("Detected environment: Google Colab")
            except Exception as colab_error:
                print("Error retrieving tokens from Google Colab:", colab_error)

        elif "/kaggle" in current_dir:  # Kaggle
            try:
                from kaggle_secrets import UserSecretsClient
                user_secrets = UserSecretsClient()
                ngrok_token = user_secrets.get_secret("NGROK_TOKEN")
                ngrok_domain = user_secrets.get_secret("NGROK_DOMAIN")
                print("Detected environment: Kaggle")
            except Exception as kaggle_error:
                print("Error retrieving tokens from Kaggle:", kaggle_error)

        else:  # Other environments
            ngrok_token = os.environ.get("NGROK_TOKEN")
            ngrok_domain = os.environ.get("NGROK_DOMAIN")
            print("Detected environment: Other (Environment Variables)")

    # If ngrok_token is still None, raise an error
    if not ngrok_token:
        raise ValueError("Ngrok token is missing. Please set the 'NGROK_TOKEN'.")

    # Optionally, check for ngrok_domain if it's required
    if not ngrok_domain:
        print("Warning: Ngrok domain is missing. Proceeding without it.")

    # For demonstration purposes, print the retrieved tokens (Remove in production)
    # print("Ngrok Token:", ngrok_token)
    # print("Ngrok Domain:", ngrok_domain)

except Exception as e:
    print("Error detecting environment or retrieving tokens:", e)


In [ ]:
# Step 3: Define Function to Set Up Ngrok Tunnel
def ngrok_tunnel(port, queue, auth_token):
    try:
        ngrok.set_auth_token(auth_token)
        url = ngrok.connect(port, domain=ngrok_domain)
        queue.put(url)
    except Exception as e:
        print("Error starting ngrok tunnel:", e)

In [ ]:

# Step 4: Start the Ngrok Tunnel
try:
    ngrok_output_queue = Queue()
    ngrok_thread = Timer(2, ngrok_tunnel, args=(8188, ngrok_output_queue, ngrok_token))
    ngrok_thread.start()
    ngrok_thread.join()
    print("Ngrok URL:", ngrok_output_queue.get())
except Exception as e:
    print("Error starting ngrok thread:", e)

In [ ]:

# Step 5: Print Tunnels (If Any)
try:
    tunnels = ngrok.get_tunnels()
    print("Active tunnels:", tunnels)
except Exception as e:
    print("Error retrieving tunnels:", e)

# Step 6: Start Code Server
try:
    !code-server --auth none --port 8188
except Exception as e:
    print("Error starting code-server:", e)